# Core Imports

In [12]:
# Generic Imports
import re
from functools import partial, cached_property
from collections import defaultdict
from itertools import combinations, chain
from ast import literal_eval

# Numeric imports
import pandas as pd
import numpy as np

# File I/O
from pathlib import Path
import csv, json, openpyxl

# Logging
from tqdm import tqdm as tqdm_text
from tqdm.notebook import tqdm as tqdm_notebook

# Typing and Subclassing
from typing import Any, Callable, ClassVar, Generator, Iterable, Optional, Union
from dataclasses import dataclass, field
from abc import ABC, abstractmethod, abstractproperty

# Cheminformatics
from rdkit import Chem
from rdkit.Chem import rdChemReactions

from openff.toolkit import ForceField
from openff.toolkit.topology import Topology, Molecule

from openforcefields.openforcefields import get_forcefield_dirs_paths
OPENFF_DIR = Path(get_forcefield_dirs_paths()[0])

# File and chemistry type definitions

In [13]:
topo_dir = Path('Topologies')
topo_dir.mkdir(exist_ok=True)

# lammps_dir = Path('LAMMPS')
lammps_dir = Path('LAMMPS')
lammps_dir.mkdir(exist_ok=True)

omm_dir = Path('OpenMM')
omm_dir.mkdir(exist_ok=True)

# Creating OpenMM and LAMMPS systems

## Generating Interchange dict

In [14]:
# specify forcefield
# ff_name = 'openff-2.0.0.offxml'
ff_name = 'openff_unconstrained-2.0.0.offxml'
ff_path = OPENFF_DIR / ff_name
forcefield = ForceField(ff_path)

# Interchange generation
success_ics = defaultdict(defaultdict)
failed_ics  = defaultdict(list)

for chem_dir in topo_dir.iterdir():
    chemistry = chem_dir.stem
    progress = tqdm_notebook([path for path in chem_dir.iterdir()]) # unpack into list for progress bar

    for sdf_path in progress:
        mol_name = sdf_path.stem
        progress.set_postfix_str(f'{chemistry} : {mol_name}')

        try:
            offmol = Molecule.from_file(sdf_path, allow_undefined_stereo=True)
            offtop = Topology.from_molecules(offmol) 
            ic = forcefield.create_interchange(offtop, charge_from_molecules=[offmol])
            success_ics[chemistry][mol_name] = ic
        except Exception as e:
            print(e)
            failed_ics[e.__class__.__name__].append(sdf_path)

for err_name, err_list in failed_ics.items():
    for sdf_path in err_list:
        sdf_path.unlink() # delete dud files

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): poly(1,4-diisocyanatobutane-co-2-(2-hydroxyethyl-phenylamino)ethanol)
Problematic atoms are:
Atom atomic num: 7, name: , idx: 23, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 22, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 24, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 35, aromatic: False, chiral: False
Atom atomic num: 7, name: , idx: 68, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 67, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 69, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 80, aromatic: False, chiral: False

Warning (not error because allow_und

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

## Defining utility functions

In [15]:
from openmm import XmlSerializer
from openmm import System, Context, State
from openmm import Integrator, Force
from openmm.app import Simulation
from openmm.unit import nanometer

from openff.interchange import Interchange
from openff.units import unit as offunit


DEFAULT_STATE_PARAMS : dict[str, bool] = {
    'getPositions'  : True,
    'getVelocities' : True,
    'getForces'     : True,
    'getEnergy'     : True,
    'getParameters' : True,
    'getParameterDerivatives' : False,
    'getIntegratorParameters' : False
}


def serialize_state_and_sys(sim : Simulation, out_dir : Path, out_name : str, state_params : dict[str, bool]=DEFAULT_STATE_PARAMS) -> None:
    '''For saving State and System info of a Simulation to disc'''
    sim_dict = {
        'system' : sim.system,
        'state' : sim.context.getState(**state_params)
    }
    
    for affix, save_data in sim_dict.items():
        save_path = out_dir / f'{out_name}_{affix}.xml'
        save_path.touch()

        with save_path.open('w') as file:
            file.write( XmlSerializer.serialize(save_data) )

def apply_state_to_context(state : State, context : Context) -> None:
    '''For applying saved State data to an existing OpenMM Simulation'''
    context.setPeriodicBoxVectors(*state.getPeriodicBoxVectors())
    context.setPositions(state.getPositions())
    context.setVelocities(state.getVelocities())
    context.setTime(state.getTime())

    context.reinitialize(preserveState=True)    

def load_openmm_system(sys_path : Path, extra_forces : Optional[Union[Force, Iterable[Force]]]=None, sep_force_grps : bool=True, remove_constrs : bool=False) -> System:
    '''Load and configure a serialized OpenMM system, with optional additional parameters'''
    assert(sys_path.suffix == '.xml')
    with sys_path.open('r') as file:
        ommsys = XmlSerializer.deserialize(file.read())

    if extra_forces: # deliberately sparse to handle both Nonetype and empty list
        for force in extra_forces: 
            ommsys.addForce(force)

    if sep_force_grps:
        for i, force in enumerate(ommsys.getForces()):
            force.setForceGroup(i)

    if remove_constrs:
        for i in range(ommsys.getNumConstraints())[::-1]: # need to remove in reverse order to avoid having prior constraints "fall back down"
            ommsys.removeConstraint(i)

    return ommsys

def create_simulation2(interchange : Interchange, integrator : Integrator, forces : Optional[Iterable[Force]]=None,
                        sep_force_grps : bool=True, remove_constrs : bool=True, combine_nonbonded_forces : bool=True) -> Simulation:
    '''Specifies configuration for an OpenMM Simulation - Interchange load alows many routes for creation'''
    openmm_sys = interchange.to_openmm(combine_nonbonded_forces=combine_nonbonded_forces) 
    openmm_top = interchange.topology.to_openmm()
    openmm_pos = interchange.positions.m_as(offunit.nanometer) * nanometer

    if forces: # deliberately sparse to handle both Nonetype and empty list
        for force in forces: 
            openmm_sys.addForce(force)

    if sep_force_grps:
        for i, force in enumerate(openmm_sys.getForces()):
            force.setForceGroup(i)

    if remove_constrs:
        for i in range(openmm_sys.getNumConstraints())[::-1]: # need to remove in reverse order to avoid having prior constraints "fall back down"
            openmm_sys.removeConstraint(i)

    simulation = Simulation(openmm_top, openmm_sys, integrator)
    simulation.context.setPositions(openmm_pos)

    return simulation

## Manually create OpenMM sims from Interchange

In [17]:
# specifying simulation and ensemble parameters
from openff.interchange.components.mdconfig import MDConfig

from openmm.app import Simulation
from openmm import NonbondedForce, CustomNonbondedForce
from openmm import MonteCarloBarostat, LangevinMiddleIntegrator

from openmm.unit import atmosphere, kelvin, nanometer
from openmm.unit import femtosecond, picosecond
from openff.units import unit as offunit

# Box sizes
BOX_VECS = np.eye(3) * 10 * nanometer

# Long-range parameters
# CUTOFF = 2.0 * nanometer
CUTOFF = 0.9 * nanometer
# CUTOFF_METHOD = NonbondedForce.NoCutoff
# CUTOFF_METHOD = NonbondedForce.CutoffNonPeriodic
CUTOFF_METHOD = NonbondedForce.CutoffPeriodic

DISPERSION = True
SWITCHING  = False

# Thermodynamic/integrator parameters
T = 300*kelvin
P = 1*atmosphere

timestep = 2*femtosecond
friction = 1*picosecond**-1

# ======================================

force_names = (
    'vdW',
    'Electrostatic',
    'vdW 1-4',
    'Electrostatic 1-4',
    'Dihedral',
    'Angle',
    'Bond'
)

# looping over all urethanes
omm_sims = defaultdict(defaultdict)
for chemistry, ic_dict in success_ics.items():
    lmp_chem_dir = lammps_dir / chemistry
    lmp_chem_dir.mkdir(exist_ok=True)
    
    omm_chem_dir = omm_dir/ chemistry
    omm_chem_dir.mkdir(exist_ok=True)

    progress = tqdm_notebook(ic_dict.items())
    for mol_name, interchange in progress:
        progress.set_postfix_str(f'{chemistry} : {mol_name}')
        omm_mol_dir = omm_chem_dir / mol_name
        omm_mol_dir.mkdir(exist_ok=True)

        if omm_mol_dir.exists() and any(omm_mol_dir.iterdir()): # skip over if dir exists and is non-empty
            continue
        
    # creating OpenMM Simulation
        progress.set_description('Building OpenMM Simulation')
        # specifying thermo/baro to determine ensemble
        integrator = LangevinMiddleIntegrator(T, friction, timestep)
        # extra_forces = [MonteCarloBarostat(P, T, baro_freq)]
        extra_forces = None

        # loading OpenMM sim components from Interchange
        interchange.box = BOX_VECS
        omm_top = interchange.topology.to_openmm()
        omm_sys = interchange.to_openmm(combine_nonbonded_forces=False)
        omm_pos = interchange.positions.m_as(offunit.nanometer)

        ## Setting box vectors for periodic forces
        omm_top.setPeriodicBoxVectors(BOX_VECS)
        omm_sys.setDefaultPeriodicBoxVectors(*BOX_VECS)

        # configuring bound Force objects
        if extra_forces:
            for force in extra_forces:
                omm_sys.addForce(force)

        ## number all forces into separate force groups for separability
        for i, force in enumerate(omm_sys.getForces()):
            force.setForceGroup(i)

        ## Add labels to default forces
        for force, name in zip(omm_sys.getForces(), force_names):
            force.setName(name)

        ## reconfiguring non-bonded forces
        ### Custom nonbonded
        # nonbond_custom = omm_sys.getForce(0)
        # assert(isinstance(nonbond_custom, CustomNonbondedForce))

        # nonbond_custom.setCutoffDistance(CUTOFF)
        # nonbond_custom.setUseSwitchingFunction(SWITCHING)
        # nonbond_custom.setNonbondedMethod(CUTOFF_METHOD)
        # nonbond_custom.setUseLongRangeCorrection(DISPERSION)
 
        # ### Default nonbonded
        # nonbond = omm_sys.getForce(1)
        # assert(isinstance(nonbond, NonbondedForce))

        # nonbond.setCutoffDistance(CUTOFF)
        # nonbond.setNonbondedMethod(CUTOFF_METHOD)
        # nonbond.setUseSwitchingFunction(SWITCHING)
        # nonbond.setUseDispersionCorrection(DISPERSION)

        # create OpenMM Simulation
        sim = Simulation(omm_top, omm_sys, integrator)
        sim.context.setPositions(omm_pos)
        omm_sims[chemistry][mol_name] = sim

    # saving OpenMM files
        progress.set_description('Generating OpenMM files')

        sdf_out_path = omm_mol_dir / f'{mol_name}_topology.sdf'
        sdf_out_path.touch()

        for mol in interchange.topology.molecules: # use OpenFF format for saving Molecules (much more convenient to work with)
            mol.to_file(str(sdf_out_path), file_format=sdf_out_path.suffix[1:])
        serialize_state_and_sys(sim, out_dir=omm_mol_dir, out_name=mol_name)

    # saving LAMMPS files
        progress.set_description('Generating LAMMPS files')
        lmp_mol_dir = lmp_chem_dir / mol_name
        lmp_mol_dir.mkdir(exist_ok=True)

        lmp_path = lmp_mol_dir / f'{mol_name}.lammps'
        lmp_in_path = lmp_mol_dir / f'{mol_name}.in'

        ### creating .lmp file
        lmp = interchange.to_lammps(lmp_path)
        mdc = MDConfig.from_interchange(interchange)
        mdc.write_lammps_input(lmp_in_path)

        ### creating .in file, replacing input file with .lmp from above
        with lmp_in_path.open('r') as in_file:
            in_file_block = in_file.read()

        in_file_block = in_file_block.replace('out.lmp', f'"{lmp_path}"')

        with lmp_in_path.open('w') as in_file:
            in_file.write(in_file_block)

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

# Evaluating LAMMPS energies

In [3]:
ENERGY_EVAL_INP = Path('in.urethane') # path to 

E_MAP = {
    'ebond'  : 'Bond',
    'eangle' : 'Angle',
    'edihed' : 'Proper Torsion',
    'eimp'   : 'Improper Torsion',
    'ecoul'  : 'Coulomb Short',
    'elong'  : 'Coulomb Long',
    'evdwl'  : 'vdW',
    'etail'  : 'Dispersion',
    'epair'  : 'Nonbonded',
    'pe'     : 'Potential',
    'ke'     : 'Kinetic',
    'etotal' : 'Total'
}

CELL_KW = ( # keywords for probing unit cell sizes and angles
    'cella',
    'cellb',
    'cellc',
    'cellalpha',
    'cellbeta',
    'cellgamma',
)

In [4]:
def get_calc_lmp_energies(lmp_block : str) -> tuple[str, list[str]]:
    '''Read which thermodynamic energy contributions will be calculated from a LAMMPS input file block'''
    ENERGY_CONTRIB_REGEX = re.compile(r'^thermo_style\s(?P<thermo_style>\b\w*?\b)\s(?P<calc_energies>.*$)')

    for line in lmp_block.split('\n'):
        if (match := re.search(ENERGY_CONTRIB_REGEX, line)):
            groups = match.groupdict()
            return groups['thermo_style'], groups['calc_energies'].split(' ')
    else:
        raise ValueError('No thermo_style energy commands found in input file')

In [35]:
import lammps
from IPython.display import clear_output


failed = defaultdict(list)
records = {}
cell_sizes = {}
for subdir in lammps_dir.iterdir():
    if subdir.is_dir():
        chemistry = subdir.name
        for mol_dir in subdir.iterdir():
            mol_name = mol_dir.stem
            lammps_file = mol_dir / f'{mol_name}.lammps'
            lammps_in   = mol_dir / f'{mol_name}.in'
            
            # craete LAMMPS wrapper and execute input calc
            with lammps.lammps() as lmp: # need to create new lammps() object instance for each run
                # lmp.commands_string( ENERGY_EVAL_STR.replace('$INP_FILE', str(lammps_file)) )
                # lmp.file(str(ENERGY_EVAL_INP))
                try:
                    lmp.file(str(lammps_in)) # read input file and calculate energies

                    ## Getting energies
                    with lammps_in.open('r') as in_file:
                        thermo_style, calc_energies = get_calc_lmp_energies(in_file.read())

                    energies = {
                        E_MAP[contrib] : lmp.get_thermo(contrib)
                            for contrib in calc_energies
                    }
                except:
                    failed[chemistry].append(mol_name)
                    continue

                ## Getting unit cell dimensions
                cell_params = {
                    cp : lmp.get_thermo(cp)
                        for cp in CELL_KW
                }

            # reformatting energies
            energies = {
                f'{contrib} (kcal/mol)' : energy # add units to labels
                    for contrib, energy in energies.items()
            }
            
            # save records for Pandas DataFrames
            records[(chemistry, mol_name)] = energies
            cell_sizes[(chemistry, mol_name)] = cell_params
            clear_output() # wipe lengthy LAMMPS printouts

In [37]:
failed

defaultdict(list,
            {'polyurethane_isocyanate': ['poly(1,6-Diisocyanatohexane-co-[8-(hydroxymethyl)-4-tricyclo[5.2.1',
              'poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-[8-(hydroxymethyl)-4-tricyclo[5.2.1',
              'poly(1,4-Diisocyanatocyclohexane-co-[8-(hydroxymethyl)-3-tricyclo[5.2.1',
              'poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-trimethylcyclohexane-co-[8-(hydroxymethyl)-4-tricyclo[5.2.1'],
             'polycarbonate_phosgene': ['poly(4-[6-(4-hydroxyphenyl)-6-bicyclo[2.2']})

In [36]:
lmp_table = pd.DataFrame.from_dict(records, 'index')
lmp_table.index.names  = ['Chemistry', 'Molecule'] # ensure index labels are labelled consistently
lmp_table.sort_values('Molecule', inplace=True)
lmp_table.to_csv(lammps_dir/f'{lammps_dir.name}_PEs.csv')

# Evaluating OpenMM energies

## Loading simulations from file

In [38]:
skip = True

# parameters
sep_force_grps : bool = True
remove_constrs : bool = False

# iterate over serialized directory tree and load
if not skip:
    omm_sims = defaultdict(defaultdict)
    for subdir in omm_dir.iterdir():
        if subdir.is_dir():
            chemistry = subdir.name
            for mol_dir in subdir.iterdir():
                mol_name = mol_dir.name

                state_file = mol_dir / f'{mol_name}_state.xml'
                sys_file   = mol_dir / f'{mol_name}_system.xml'
                top_file   = mol_dir / f'{mol_name}_topology.sdf'

                offmol = Molecule.from_file(top_file)
                offtop = Topology.from_molecules(offmol)
                
                integrator = LangevinMiddleIntegrator(T, friction, timestep)
                # extra_forces = [MonteCarloBarostat(P, T, baro_freq)]
                extra_forces = None

                # load and configure System
                omm_top = offtop.to_openmm()
                omm_sys = load_openmm_system(
                    sys_file,
                    extra_forces=extra_forces,
                    sep_force_grps=sep_force_grps,
                    remove_constrs=remove_constrs
                )

                # putting it all together into a Simulation
                sim = Simulation(
                    topology=omm_top,
                    system=omm_sys,
                    integrator=integrator,
                    state=state_file
                )
                omm_sims[chemistry][mol_name] = sim

## Evaluating starting structure energies

In [39]:
from openmm.unit import kilojoule_per_mole, kilocalorie_per_mole

NULL_ENERGY = 0.0*kilojoule_per_mole
PRECISION : int = 4

data_dicts = []
for chemistry, mol_dict in omm_sims.items():
    progress = tqdm_notebook(mol_dict.items())
    for mol_name, sim in progress:
        progress.set_postfix_str(f'{chemistry} : {mol_name}')
        
        # extract total and component energies from OpenMM force groups
        data_dict = {
            'Chemistry' : chemistry,
            'Molecule'  : mol_name
        }
        omm_energies = {}

        ## Total Potential
        overall_state = sim.context.getState(getEnergy=True) # get total potential energy
        PE = overall_state.getPotentialEnergy()
        omm_energies['Potential'] = PE

        ## Total Kinetic (to verify no integration is being done)
        KE = overall_state.getKineticEnergy()
        omm_energies['Kinetic'] = KE
        assert(KE == NULL_ENERGY)

        ## Individual force contributions
        for i, force in enumerate(sim.system.getForces()):
            state = sim.context.getState(getEnergy=True, groups={i})
            omm_energies[force.getName()] = state.getPotentialEnergy()

        # reformat to desired units and precision
        omm_energies_kcal = {}
        for contrib_name, energy_kj in omm_energies.items():
            energy_kcal = energy_kj.in_units_of(kilocalorie_per_mole)
            omm_energies_kcal[f'{contrib_name} ({energy_kcal.unit.get_symbol()})'] = round(energy_kcal._value, PRECISION)

        # compile data
        data_dict = {**data_dict, **omm_energies_kcal}
        data_dicts.append(data_dict)

omm_table = pd.DataFrame.from_records(data_dicts)
omm_table.sort_values('Molecule', inplace=True)
omm_table.set_index(['Chemistry', 'Molecule'], inplace=True)
omm_table.to_csv(omm_dir / f'{omm_dir.name}_PEs.csv')

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

# Comparing energies

## Loading energy tables and comparing contributions

In [41]:
pd.options.display.float_format = '{:.4f}'.format # disable scientific notation

@dataclass
class TableFormats:
    table_key : str
    sum_terms : dict[str, list[str]]
    del_terms : list[str]

omm_formats = TableFormats(
    table_key = omm_dir.stem,
    sum_terms = {
        'vdW (kcal/mol)' : ['vdW (kcal/mol)', 'vdW 1-4 (kcal/mol)'],
        'Coulomb (kcal/mol)' : ['Electrostatic (kcal/mol)', 'Electrostatic 1-4 (kcal/mol)']
    },
    del_terms = ['Kinetic (kcal/mol)']
)

lmp_formats = TableFormats(
    table_key = lammps_dir.stem,
    sum_terms = {
        'vdW (kcal/mol)' : ['vdW (kcal/mol)', 'Dispersion (kcal/mol)'],
        'Dihedral (kcal/mol)' : ['Proper Torsion (kcal/mol)', 'Improper Torsion (kcal/mol)'],
        'Coulomb (kcal/mol)' : ['Coulomb Short (kcal/mol)', 'Coulomb Long (kcal/mol)']
    },
    del_terms = ['Nonbonded (kcal/mol)']
)

# apply reformatting to respective tables
for fmt in (omm_formats, lmp_formats):
    table_in_path  = Path(fmt.table_key) / f'{fmt.table_key}_PEs.csv'
    table_out_path = Path(fmt.table_key) / f'{fmt.table_key}_PEs_processed.csv'
    table = pd.read_csv(table_in_path, index_col=(0, 1)).sort_index(axis=1)

    # combine selected terms
    for combined_contrib, contribs in fmt.sum_terms.items():
        new_term = sum(
            table[contrib]
                for contrib in contribs
        ) # merge contributions into a single new named term
        table.drop(columns=contribs, inplace=True) # clear contributions
        table[combined_contrib] = new_term # done after drop to ensure name clashes don;t result in extra deletion
    
    # delete redundant terms
    for del_contrib in fmt.del_terms:
        table.drop(columns=[del_contrib], inplace=True) # clear contributions

    globals()[f'{fmt.table_key.lower()}_table'] = table
    table.to_csv(table_out_path)

In [42]:
openmm_table

Angle (kcal/mol)  \
Chemistry               Molecule                                                               
polycarbonate_phosgene  poly(1,1,1',1'-tetramethyl-3,3'-spirobi[2H-inde...          187.1410   
polyurethane_isocyanate poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-Bu...          132.1359   
                        poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-[4...          171.8093   
                        poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-[8...          243.2126   
                        poly(1,4-Diisocyanatocyclohexane-co-[8-(hydroxy...          225.4009   
...                                                                                      ...   
polyamide               poly(HEXANE-1,6-DIAMINE-co-Dodecanedioic acid)              171.6889   
                        poly(HEXANE-1,6-DIAMINE-co-hexanedioic acid)                115.0001   
                        poly(benzene-1,3-diamine-co-1-(4-carboxyphenyl)...          179.2985   
                        poly(benzene-1,4-diamine-co-1-(4-carboxyphenyl)...          179.3701   
                        poly(naphthalene-1,5-diamine-co-1-(4-carboxyphe...          202.2276   

                                                                            Bond (kcal/mol)  \
Chemistry               Molecule                                                              
polycarbonate_phosgene  poly(1,1,1',1'-tetramethyl-3,3'-spirobi[2H-inde...         173.0979   
polyurethane_isocyanate poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-Bu...         149.6479   
                        poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-[4...         153.7217   
                        poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-[8...         155.0413   
                        poly(1,4-Diisocyanatocyclohexane-co-[8-(hydroxy...         153.5499   
...                                                                                     ...   
polyamide               poly(HEXANE-1,6-DIAMINE-co-Dodecanedioic acid)             160.7975   
                        poly(HEXANE-1,6-DIAMINE-co-hexanedioic acid)               157.1983   
                        poly(benzene-1,3-diamine-co-1-(4-carboxyphenyl)...         231.7064   
                        poly(benzene-1,4-diamine-co-1-(4-carboxyphenyl)...         231.2555   
                        poly(naphthalene-1,5-diamine-co-1-(4-carboxyphe...         257.4496   

                                                                            Dihedral (kcal/mol)  \
Chemistry               Molecule                                                                  
polycarbonate_phosgene  poly(1,1,1',1'-tetramethyl-3,3'-spirobi[2H-inde...              33.8524   
polyurethane_isocyanate poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-Bu...              31.4386   
                        poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-[4...              47.3881   
                        poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-[8...              86.8057   
                        poly(1,4-Diisocyanatocyclohexane-co-[8-(hydroxy...              78.1473   
...                                                                                         ...   
polyamide               poly(HEXANE-1,6-DIAMINE-co-Dodecanedioic acid)                  51.1838   
                        poly(HEXANE-1,6-DIAMINE-co-hexanedioic acid)                    49.8895   
                        poly(benzene-1,3-diamine-co-1-(4-carboxyphenyl)...              24.6403   
                        poly(benzene-1,4-diamine-co-1-(4-carboxyphenyl)...              35.9005   
                        poly(naphthalene-1,5-diamine-co-1-(4-carboxyphe...              37.7482   

                                                                            Potential (kcal/mol)  \
Chemistry               Molecule                                                                   
polycarbonate_phosgene  poly(1,1,1',1'-tetramethyl-3,3'-spirobi[2H-inde...              310.1291   
pol

In [43]:
lammps_table

Angle (kcal/mol)  \
Chemistry               Molecule                                                               
polycarbonate_phosgene  poly(1,1,1',1'-tetramethyl-3,3'-spirobi[2H-inde...          187.1410   
polyurethane_isocyanate poly(1,10-diisocyanatodecane-co-Decane-1,10-diol)           208.7093   
                        poly(1,10-diisocyanatodecane-co-octadec-9-ene-1...          278.7890   
                        poly(1,12-diisocyanatooctadec-9-ene-co-Decane-1...          276.4750   
                        poly(1,18-diisocyanatooctadecane-co-Decane-1,10...          289.7186   
...                                                                                      ...   
polyester               poly([4-(hydroxymethyl)cyclohexyl]methanol-co-T...          128.3913   
                        poly([4-(hydroxymethyl)cyclohexyl]methanol-co-c...          156.6520   
polyamide               poly(benzene-1,3-diamine-co-1-(4-carboxyphenyl)...          179.2986   
                        poly(benzene-1,4-diamine-co-1-(4-carboxyphenyl)...          179.3702   
                        poly(naphthalene-1,5-diamine-co-1-(4-carboxyphe...          202.2279   

                                                                            Bond (kcal/mol)  \
Chemistry               Molecule                                                              
polycarbonate_phosgene  poly(1,1,1',1'-tetramethyl-3,3'-spirobi[2H-inde...         173.0979   
polyurethane_isocyanate poly(1,10-diisocyanatodecane-co-Decane-1,10-diol)          153.5384   
                        poly(1,10-diisocyanatodecane-co-octadec-9-ene-1...         157.9173   
                        poly(1,12-diisocyanatooctadec-9-ene-co-Decane-1...         157.5051   
                        poly(1,18-diisocyanatooctadecane-co-Decane-1,10...         158.0609   
...                                                                                     ...   
polyester               poly([4-(hydroxymethyl)cyclohexyl]methanol-co-T...         166.1781   
                        poly([4-(hydroxymethyl)cyclohexyl]methanol-co-c...         143.4659   
polyamide               poly(benzene-1,3-diamine-co-1-(4-carboxyphenyl)...         231.7062   
                        poly(benzene-1,4-diamine-co-1-(4-carboxyphenyl)...         231.2554   
                        poly(naphthalene-1,5-diamine-co-1-(4-carboxyphe...         257.4496   

                                                                            Potential (kcal/mol)  \
Chemistry               Molecule                                                                   
polycarbonate_phosgene  poly(1,1,1',1'-tetramethyl-3,3'-spirobi[2H-inde...              294.0470   
polyurethane_isocyanate poly(1,10-diisocyanatodecane-co-Decane-1,10-diol)           4521652.1293   
                        poly(1,10-diisocyanatodecane-co-octadec-9-ene-1...         19997151.5275   
                        poly(1,12-diisocyanatooctadec-9-ene-co-Decane-1...          4523258.5493   
                        poly(1,18-diisocyanatooctadecane-co-Decane-1,10...          4499343.3532   
...                                                                                          ...   
polyester               poly([4-(hydroxymethyl)cyclohexyl]methanol-co-T...     628317668918.2488   
                        poly([4-(hydroxymethyl)cyclohexyl]methanol-co-c...     628338176069.1810   
polyamide               poly(benzene-1,3-diamine-co-1-(4-carboxyphenyl)...            29712.7904   
                        poly(benzene-1,4-diamine-co-1-(4-carboxyphenyl)...              396.4964   
                        poly(naphthalene-1,5-diamine-co-1-(4-carboxyphe...              762.7979   

                                                                              vdW (kcal/mol)  \
Chemistry               Molecule                                                               
polycarbonate_phosgene  poly(1,1,1',1'-tetramethyl-3,3'-spirobi[2H-inde...           38.8089   
po

In [44]:
diff = openmm_table - lammps_table
diff

Angle (kcal/mol)  \
Chemistry               Molecule                                                               
polyamide               poly(1,5-bis(3-aminophenyl)penta-1,4-dien-3-one...           -0.0002   
                        poly(1,5-bis(3-aminophenyl)penta-1,4-dien-3-one...            0.0000   
                        poly(3,3-bis[4-[4-(4-aminophenyl)-2-(trifluorom...           -0.0000   
                        poly(4-(4-amino-2-methylphenyl)-3-methylaniline...           -0.0004   
                        poly(4-(4-aminophenoxy)aniline-co-1-(4-carboxyp...            0.0005   
...                                                                                      ...   
polyurethane_isocyanate poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...               NaN   
                        poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...           -0.0000   
                        poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...               NaN   
                        poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...               NaN   
                        poly(5-isocyanato-2-[4-[6-[4-(5-isocyanato-1,3-...           -0.0005   

                                                                            Bond (kcal/mol)  \
Chemistry               Molecule                                                              
polyamide               poly(1,5-bis(3-aminophenyl)penta-1,4-dien-3-one...           0.0021   
                        poly(1,5-bis(3-aminophenyl)penta-1,4-dien-3-one...           0.0007   
                        poly(3,3-bis[4-[4-(4-aminophenyl)-2-(trifluorom...           0.0002   
                        poly(4-(4-amino-2-methylphenyl)-3-methylaniline...          -0.0002   
                        poly(4-(4-aminophenoxy)aniline-co-1-(4-carboxyp...           0.0003   
...                                                                                     ...   
polyurethane_isocyanate poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...              NaN   
                        poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...           0.0001   
                        poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...              NaN   
                        poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...              NaN   
                        poly(5-isocyanato-2-[4-[6-[4-(5-isocyanato-1,3-...          -0.0003   

                                                                            Coulomb (kcal/mol)  \
Chemistry               Molecule                                                                 
polyamide               poly(1,5-bis(3-aminophenyl)penta-1,4-dien-3-one...              0.1405   
                        poly(1,5-bis(3-aminophenyl)penta-1,4-dien-3-one...             -1.6572   
                        poly(3,3-bis[4-[4-(4-aminophenyl)-2-(trifluorom...              0.9282   
                        poly(4-(4-amino-2-methylphenyl)-3-methylaniline...             -0.3292   
                        poly(4-(4-aminophenoxy)aniline-co-1-(4-carboxyp...              1.9217   
...                                                                                        ...   
polyurethane_isocyanate poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...                 NaN   
                        poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...             -1.7398   
                        poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...                 NaN   
                        poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...                 NaN   
                        poly(5-isocyanato-2-[4-[6-[4-(5-isocyanato-1,3-...              0.1605   

                                                                            Dihedral (kcal/mol)  \
Chemistry               Molecule                                                                  
polyamide               poly(1,5-bis(3-aminophenyl)penta-1,4-dien-3-one...               0.0001   
                   

In [45]:
common_cols = ['Angle (kcal/mol)', 'Bond (kcal/mol)']# 'Torsion (kcal/mol)']

omm_redux = omm_table.drop(columns=common_cols)
lmp_redux = lmp_table.drop(columns=common_cols)

In [46]:
omm_table[common_cols] - lmp_table[common_cols]

Angle (kcal/mol)  \
Chemistry               Molecule                                                               
polyamide               poly(1,5-bis(3-aminophenyl)penta-1,4-dien-3-one...           -0.0002   
                        poly(1,5-bis(3-aminophenyl)penta-1,4-dien-3-one...            0.0000   
                        poly(3,3-bis[4-[4-(4-aminophenyl)-2-(trifluorom...           -0.0000   
                        poly(4-(4-amino-2-methylphenyl)-3-methylaniline...           -0.0004   
                        poly(4-(4-aminophenoxy)aniline-co-1-(4-carboxyp...            0.0005   
...                                                                                      ...   
polyurethane_isocyanate poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...               NaN   
                        poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...           -0.0000   
                        poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...               NaN   
                        poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...               NaN   
                        poly(5-isocyanato-2-[4-[6-[4-(5-isocyanato-1,3-...           -0.0005   

                                                                            Bond (kcal/mol)  
Chemistry               Molecule                                                             
polyamide               poly(1,5-bis(3-aminophenyl)penta-1,4-dien-3-one...           0.0021  
                        poly(1,5-bis(3-aminophenyl)penta-1,4-dien-3-one...           0.0007  
                        poly(3,3-bis[4-[4-(4-aminophenyl)-2-(trifluorom...           0.0002  
                        poly(4-(4-amino-2-methylphenyl)-3-methylaniline...          -0.0002  
                        poly(4-(4-aminophenoxy)aniline-co-1-(4-carboxyp...           0.0003  
...                                                                                     ...  
polyurethane_isocyanate poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...              NaN  
                        poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...           0.0001  
                        poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...              NaN  
                        poly(5-Isocyanato-1-(isocyanatomethyl)-1,3,3-tr...              NaN  
                        poly(5-isocyanato-2-[4-[6-[4-(5-isocyanato-1,3-...          -0.0003  

[134 rows x 2 columns]

In [47]:
omm_redux

Potential (kcal/mol)  \
Chemistry               Molecule                                                                   
polycarbonate_phosgene  poly(1,1,1',1'-tetramethyl-3,3'-spirobi[2H-inde...              310.1291   
polyurethane_isocyanate poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-Bu...         22642166.0386   
                        poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-[4...              519.6899   
                        poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-[8...            76195.3768   
                        poly(1,4-Diisocyanatocyclohexane-co-[8-(hydroxy...        958542379.9330   
...                                                                                          ...   
polyamide               poly(HEXANE-1,6-DIAMINE-co-Dodecanedioic acid)           1327945428.7096   
                        poly(HEXANE-1,6-DIAMINE-co-hexanedioic acid)             1328006518.6564   
                        poly(benzene-1,3-diamine-co-1-(4-carboxyphenyl)...            29713.4561   
                        poly(benzene-1,4-diamine-co-1-(4-carboxyphenyl)...              398.9589   
                        poly(naphthalene-1,5-diamine-co-1-(4-carboxyphe...              763.7421   

                                                                            Kinetic (kcal/mol)  \
Chemistry               Molecule                                                                 
polycarbonate_phosgene  poly(1,1,1',1'-tetramethyl-3,3'-spirobi[2H-inde...              0.0000   
polyurethane_isocyanate poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-Bu...              0.0000   
                        poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-[4...              0.0000   
                        poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-[8...              0.0000   
                        poly(1,4-Diisocyanatocyclohexane-co-[8-(hydroxy...              0.0000   
...                                                                                        ...   
polyamide               poly(HEXANE-1,6-DIAMINE-co-Dodecanedioic acid)                  0.0000   
                        poly(HEXANE-1,6-DIAMINE-co-hexanedioic acid)                    0.0000   
                        poly(benzene-1,3-diamine-co-1-(4-carboxyphenyl)...              0.0000   
                        poly(benzene-1,4-diamine-co-1-(4-carboxyphenyl)...              0.0000   
                        poly(naphthalene-1,5-diamine-co-1-(4-carboxyphe...              0.0000   

                                                                            vdW (kcal/mol)  \
Chemistry               Molecule                                                             
polycarbonate_phosgene  poly(1,1,1',1'-tetramethyl-3,3'-spirobi[2H-inde...         -0.0077   
polyurethane_isocyanate poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-Bu...   22641747.6047   
                        poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-[4...         48.6941   
                        poly(1,3-Bis(isocyanatomethyl)cyclohexane-co-[8...      75136.4619   
                        poly(1,4-Diisocyanatocyclohexane-co-[8-(hydroxy...  958541950.2780   
...                                                                                    ...   
polyamide               poly(HEXANE-1,6-DIAMINE-co-Dodecanedioic acid)     1327945055.4420   
                        poly(HEXANE-1,6-DIAMINE-co-hexanedioic acid)       1328005996.1722   
                        poly(benzene-1,3-diamine-co-1-(4-carboxyphenyl)...      29523.3001   
                        poly(benzene-1,4-diamine-co-1-(4-carboxyphenyl)...         70.0686   
                        poly(naphthalene-1,5-diamine-co-1-(4-carboxyphe...        366.3478   

                                                                            Electrostatic (kcal/mol)  \
Chemistry               Molecule                                                                       
polycarbonate_phosgene  poly(1,1,1',1'-tetramethyl-3,3'-spirobi[2H-in

## Evaluating energies with drivers

In [ ]:
from openff.interchange.drivers.openmm import get_openmm_energies, _get_openmm_energies
from openff.interchange.drivers.lammps import get_lammps_energies, _get_lammps_energies,  _find_lammps_executable
from openff.units.openmm import to_openmm as openff_units_to_openmm

In [ ]:
{
    contrib : openff_units_to_openmm(value).in_units_of(kilocalorie_per_mole)
        for contrib, value in get_openmm_energies(interchange, detailed=True, combine_nonbonded_forces=False).energies.items()
}

In [ ]:
get_lammps_energies(interchange).energies

## Comparing ParmEd energy decomposition to native OpenMM force-group-based decomposition

In [ ]:
import parmed
from openmm.openmm import Force

NULL_ENERGY = 0.0*kilojoule_per_mole

sim = omm_sims['urethane']['urethane_41']
# assign and initialize unique force groups for simulation
for i, force in enumerate(sim.system.getForces()):
    force.setForceGroup(i)
    # print(force.getName(), force.getForceGroup())
sim.context.reinitialize(preserveState=True) # need to reinitialize to get force labelling changes to "stick"

# energies from OpenMM force groups
print('\nOpenMM:')
print('='*30)
omm_energies = {}

## extract total energies for state
overall_state = sim.context.getState(getEnergy=True) # get total potential energy
PE = overall_state.getPotentialEnergy()
omm_energies['Total Potential Energy'] = PE

KE = overall_state.getKineticEnergy()
assert(KE == NULL_ENERGY)

for i, force in enumerate(sim.system.getForces()):
    state = sim.context.getState(getEnergy=True, groups={i})
    force_name = force.getName().removesuffix('Force')
    pe = state.getPotentialEnergy()

    omm_energies[force_name] = pe
    print(f'{force_name} : {pe}')

## converting name to match with ParmEd for comparison
namemap = {
    'Nonbonded' : 'bond',
    'PeriodicTorsion' : 'angle',
    'HarmonicAngle' : 'dihedral',
    'HarmonicBond' : 'urey_bradley',
    'Total Potential Energy' : 'total'
}
compat_omm_energies = {
    namemap[contrib] : energy
        for contrib, energy in omm_energies.items()
}

total = sum(omm_energies.values(), start=NULL_ENERGY) # need "seed" to have Quantity datatype to sum
print(f'{general.GREEK_UPPER["delta"]}E_contrib: ', PE - total)

# ParmEd energy decomposition
print('\nParmEd:')
print('='*30)
parm_energies = {}
parm_struct = parmed.openmm.load_topology(sim.topology, sim.system)
for contrib, energy_val in parmed.openmm.energy_decomposition(parm_struct, sim.context).items():
    parm_energies[contrib] = energy = energy_val*kilocalorie_per_mole # assign proper units
    print(contrib, energy.in_units_of(kilojoule_per_mole))